In [ ]:
# Tensorflow to ONNX conversion is supported through the tf2onnx converter.
import onnxmltools
import tensorflow as tf
import tf2onnx

To convert a TensorFlow model, tf2onnx prefers a frozen TensorFlow graph with specified inputs and outputs for the graph, alongside a source checkpoint file. More details on usage can be found in the [tf2onnx](https://github.com/onnx/tensorflow-onnx#usage) repository.

```
python3 -m tf2onnx.convert 
    --input SOURCE_GRAPHDEF_PB
    --graphdef SOURCE_GRAPHDEF_PB
    --checkpoint SOURCE_CHECKPOINT
    --saved-model SOURCE_SAVED_MODEL
    [--inputs GRAPH_INPUTS]
    [--outputs GRAPH_OUTPUS]
    [--inputs-as-nchw inputs_provided_as_nchw]
    [--target TARGET]
    [--output TARGET_ONNX_GRAPH]
    [--target TARGET]
    [--continue_on_error]
    [--verbose]
    [--custom-ops list-of-custom-ops]
    [--opset OPSET]
    [--fold_const]
```

In [ ]:
# savedmodel is the name of the directory from your tf.saved_model output
# model.onnx is the name of your desired output model

!python3 -m tf2onnx.convert \
        --saved-model savedmodel \
        --output model.onnx \
        --target rs6 \
        --fold_const \
        --verbose

In [ ]:
# model.pb is the name of your checkpoint
# graph_inputs are the names of your input nodes
# graph_outputs are the names of your output nodes 
# model.onnx is the name of your desired output model

# If you know your graph input / output node names, then you can use the freeze_graph method instead
# input and output node names can be discovered through Netron, tensorboard, or the TF summarize_graph tool

!python3 -m tf2onnx.convert \
        --input model.pb \
        --inputs GRAPH_INPUTS \
        --outputs GRAPH_OUTPUTS \
        --output model.onnx \
        --fold_const \
        --verbose

Alternatively, you could use a model from an active Tensorflow session in Python, freeze the graph, and then convert it, as demonstrated below.

In [ ]:
# Replace this with your desired input TF model name
input_model_name = "tf_model"

# Replace this with your desired output ONNX model name
output_onnx_model = "model.onnx"

In [ ]:
with tf.Session() as sess:
    # Note: this is a simple example Tensorflow model
    x = tf.placeholder(tf.float32, [2, 3], name="input")
    x_ = tf.add(x, x)
    _ = tf.identity(x_, name="output")
    onnx_graph = tf2onnx.tfonnx.process_tf_graph(sess.graph, input_names=["input:0"], output_names=["output:0"])
    onnx_model = onnx_graph.make_model(input_model_name)

In [ ]:
# Save as protobuf

onnxmltools.utils.save_model(onnx_model, output_onnx_model)